## Custom Data Loader

## 1. Settings
### 1) Import required libraries

In [1]:
import torch
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms

### 2) Hyperparameter

In [3]:
batch_size = 2

## 3) ImageFolder & DataLoaderm

In [7]:
img_dir = "./images"
img_data = dset.ImageFolder(img_dir, transforms.Compose([
            transforms.Scale(256),
            transforms.RandomSizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            ]))

img_batch = data.DataLoader(img_data, batch_size=batch_size,
                            shuffle=True,drop_last=True)

C:\Users\user\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
C:\Users\user\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:563: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  "please use transforms.RandomResizedCrop instead.")


## 4) Test

In [8]:
for image,label in img_batch:
    print(image.size(),label)

torch.Size([2, 3, 224, 224]) tensor([0, 1])
torch.Size([2, 3, 224, 224]) tensor([1, 1])


## VGGNet

# [VGGNet Implementation](https://arxiv.org/pdf/1409.1556.pdf)

- 2014 ILSVRC 2nd place
- VGG-16
- Convolution layer
- Maxpooling layer
- Fully connected layer

![alt text](./networks/vgg16.png)

## 1. Settings

### 1) Import required libraries

In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

### 2) Hyperparameter

In [12]:
batch_size= 1
learning_rate = 0.0002
epoch = 100

## 2. Data Loader

In [13]:
img_dir = "./images"
img_data = dset.ImageFolder(img_dir, transforms.Compose([
            transforms.Scale(256),
            transforms.RandomSizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            ]))

img_batch = data.DataLoader(img_data, batch_size=batch_size,
                            shuffle=True, num_workers=2)

C:\Users\user\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
C:\Users\user\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:563: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  "please use transforms.RandomResizedCrop instead.")


## 3. Model

### 1) Basic Blocks

In [17]:
def conv_2_block(in_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2))
    
    return model

def conv_3_block(in_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

### 3) VGG Model

In [20]:
class VGG(nn.Module):
    
    def __init__(self, base_dim, num_classes=2):
        super(VGG, self).__init__()
        self.feature = nn.Sequential(
            conv_2_block(3,base_dim),
            conv_2_block(base_dim, 2*base_dim),
            conv_3_block(2*base_dim, 4*base_dim),
            conv_3_block(4*base_dim, 8*base_dim),
            conv_3_block(8*base_dim, 8*base_dim))
        
        self.fc_layer = nn.Sequential(
            nn.Linear(8*base_dim * 7 * 7, 100),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(100,20),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(20,num_classes)
            )
        
    def forward(self, x):
        x = self.feature(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x
    
model = VGG(base_dim = 64)

for i in model.named_children():
    print(i)

('feature', Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Seq

## 4. Optimizer & Loss


In [21]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

## Train

In [23]:
for i in range(epoch):
    for img, label in img_batch:
        img = Variable(img)
        label = Variable(label)
        
        optimizer.zero_grad()
        output = model(img)
        loss = loss_func(output, label)
        loss.backward()
        optimizer.step()
        
    if i % 10 == 0:
        print(loss)

tensor(0.8593, grad_fn=<NllLossBackward>)
tensor(0.8738, grad_fn=<NllLossBackward>)
tensor(0.9593, grad_fn=<NllLossBackward>)
tensor(0.5670, grad_fn=<NllLossBackward>)
tensor(0.5539, grad_fn=<NllLossBackward>)
tensor(0.5694, grad_fn=<NllLossBackward>)
tensor(0.5376, grad_fn=<NllLossBackward>)
tensor(0.8694, grad_fn=<NllLossBackward>)
tensor(0.5685, grad_fn=<NllLossBackward>)
tensor(0.5595, grad_fn=<NllLossBackward>)
